In [3]:
import os
import pickle

import pandas as pd
from multipy.fdr import lsu
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.weightstats import ttest_ind

In [4]:
def read_data(data_path):
    with open(os.path.join(data_path, "dataset.pkl"), "rb") as f:
        data = pickle.load(f)
        Y = data["pheno"].iloc[:, 1:]
        X_gpa = data["X_gpa"]
        X_snps = data["X_snps"]
        X_genexp = data["X_genexp"]

    return X_gpa, X_snps, X_genexp, Y

In [5]:
X_gpa, X_snps, X_genexp, Y = read_data("")
y = Y["Colistin"].to_numpy()

In [6]:
res_idx = np.where(y==1)
sens_idx = np.where(y==0)

In [7]:
gpa_pval = np.apply_along_axis(lambda x: proportions_ztest((sum(x[res_idx]), sum(x[sens_idx])),
                                                           (len(x[res_idx]), len(x[sens_idx])))[1], 0, X_gpa)

/home/vadim/miniconda3/envs/app-stat/lib/python3.10/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std


In [10]:
snps_pval = np.apply_along_axis(lambda x: proportions_ztest((sum(x[res_idx]), sum(x[sens_idx])),
                                                            (len(x[res_idx]), len(x[sens_idx])))[1], 0, X_snps)

In [13]:
genexp_pval = np.apply_along_axis(lambda x: ttest_ind(x[res_idx], x[sens_idx])[1], 0, X_genexp)

In [40]:
columns = pd.MultiIndex.from_tuples(list(zip(["# régresseurs"], ["total"])) +
                                    list(zip(["Niveau $\\alpha$"]*3, [.1, .05, .01])))
df = pd.DataFrame([[len(gpa_pval), sum(lsu(gpa_pval, .1)), sum(lsu(gpa_pval, .05)), sum(lsu(gpa_pval, .01))],
                   [len(snps_pval), sum(lsu(snps_pval, .1)), sum(lsu(snps_pval, .05)), sum(lsu(snps_pval, .01))],
                   [len(genexp_pval), sum(lsu(genexp_pval, .1)), sum(lsu(genexp_pval, .05)), sum(lsu(genexp_pval, .01))]],
                  index=["gpa", "snps", "genexp"],
                  columns=columns)
df

# régresseurs Niveau $\alpha$            
               total             0.1  0.05  0.01
gpa            16005             575   374   157
snps           72236            5258  3053  1233
genexp          6026              96    77    56

In [48]:
df.to_latex(buf="rapport/tables/mul_test.tex",
            float_format="%.0f",
            bold_rows=True,
            escape=True,
            caption="Nombre de covariables sélectionnées pour différents niveaux $\\alpha$.",
            label="tab:mul_test",
            position="H")
with open("rapport/tables/mul_test.tex", "r") as f:
    text = f.read().replace("\\$\\textbackslash alpha\\$", "$\\alpha$")
with open("rapport/tables/mul_test.tex", "w") as f:
    f.write(text)

/tmp/ipykernel_91518/917976342.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf="rapport/tables/mul_test.tex",
